# Ancilla Evolution - *After the System Thermalizes*

In [6]:
%matplotlib inline
import copy
from qutip import *
import numpy as np
import math, cmath
import matplotlib.pyplot as plt
from tqdm import tqdm

import utilities as use
from stateobj import QAncilla

from IPython.display import Markdown, display

In [7]:
TIMESTEPS = 1000
TIMEDELTA = 1e-02
OMEGA = 10

## Thermal System

In [8]:
SIM_ID = '40D'
system = qload(f'objects/steadystate_meq_{SIM_ID}')

Loaded Qobj object:
Quantum object: dims = [[40], [40]], shape = (40, 40), type = oper, isHerm = True



## Ancilla State

In [9]:
alpha = complex(1/math.sqrt(2), 0)
beta = complex(1/math.sqrt(2), 0)
phi = np.pi/3
eta = QAncilla(alpha, beta, phi)
# Stampa informazioni sull'Ancella
display(eta)

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[0.5  +0.j         0.   +0.j         0.   +0.j        ]
 [0.   +0.j         0.25 +0.j         0.125+0.21650635j]
 [0.   +0.j         0.125-0.21650635j 0.25 +0.j        ]]

## Interaction

In [11]:
# System Opeators
ap_int = tensor(system.ap, qeye(3))
am_int = tensor(system.am, qeye(3))
# Ancilla Operators 
bp_int = tensor(qeye(system.dimentions), eta.sigmaminus)
bm_int = tensor(qeye(system.dimentions), eta.sigmaplus)
# INTERACTION
V = OMEGA * ( ap_int*bp_int + am_int*bm_int )

## Evolution

In [15]:
total_system = tensor(system, eta)
total_system = use.evolve(total_system, V, TIMEDELTA)
eta_new = total_system.ptrace(1)
eta_new

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[0.50000002+0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.25000001+0.j         0.125     +0.14316943j]
 [0.        +0.j         0.125     -0.14316943j 0.25000001+0.j        ]]

### Theoretical Evolution
$$ \eta' = \eta + \Delta t\left[
   \langle A_+A_- \rangle\left(A_-\rho A_+ -\frac{1}{2}[A_+A_-; \eta]\right) +
   \langle A_-A_+ \rangle\left(A_+\rho A_- -\frac{1}{2}[A_-A_+; \eta]\right)
    \right] $$

In [19]:
print(f"<A+> = {expect(system.ap, system).real}")
print(f"<A+> = {expect(system.ap, system).real}")

<A+> = 0.0
<A+> = 0.0


### *Moments*

In [21]:
first_moments = expect([system.ap, system.am], system)
print(f"First Moments:\t\tComputed\tTheoretical\n"
      f"\tA- :\t\t{first_moments[0].real:.4f}\t\t0\n"
      f"\tA+ :\t\t{first_moments[1].real:.4f}\t\t0\n")
second_moments = expect([system.ap*system.ap, system.am*system.am], system)
print(f"Second Moments:\t\tComputed\tTheoretical\n"
      f"\tA-A- :\t\t{second_moments[0].real:.4f}\t\t0\n"
      f"\tA+A+ :\t\t{second_moments[1].real:.4f}\t\t0\n")


First Moments:		Computed	Theoretical
	A- :		0.0000		0
	A+ :		0.0000		0

Second Moments:		Computed	Theoretical
	A-A- :		0.0000		0
	A+A+ :		0.0000		0



### Cross Correlations
$\langle a a^\dagger\rangle = \frac{1}{Z}\sum_n (n+1)\left( \gamma_\beta / \gamma_\alpha \right)^n 
=1+\frac{1}{Z}\sum_n n\left( \gamma_\beta / \gamma_\alpha \right)^n 
\longrightarrow 1+\frac{\gamma_\beta / \gamma_\alpha}{1 - \gamma_\beta / \gamma_\alpha}
$

$\langle a^\dagger a \rangle = \frac{1}{Z}\sum_n n\left( \gamma_\beta / \gamma_\alpha \right)^n 
\longrightarrow \frac{\gamma_\beta / \gamma_\alpha}{1 - \gamma_\beta / \gamma_\alpha}
$

In [22]:
system

Quantum object: dims = [[40], [40]], shape = (40, 40), type = oper, isherm = True
Qobj data =
[[3.35262508e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 4.47003347e-06 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 5.95991169e-06 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.40626418e-01
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  1.87501890e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 2.50002520e-01]]

In [19]:
cross_corr = expect([rho.am*rho.ap, rho.ap*rho.am], rho)
series = (1-eta.factor)**2
series = eta.factor / series
print(f"Cross-correlations:\tComputed\tTheoretical\n"
      f"\tA-A+ :\t\t{cross_corr[0]:>8.7}\t{1 + 1/rho.partition*series:0.7}\n"
      f"\tA+A- :\t\t{cross_corr[1]:>8.7}\t{1/rho.partition*series:>0.7}")

Cross-correlations:	Computed	Theoretical
	A-A+ :		 27.0003	4.00003
	A+A- :		 36.0004	3.00003


In [26]:
eta_new = eta
for t in tqdm(range(1000)):
    first_term = cross_corr[0]*(eta.sigmaminus*eta*eta.sigmaplus-0.5*commutator(eta.sigmaplus*eta.sigmaminus, eta))
    second_term = cross_corr[1]*(eta.sigmaplus*eta*eta.sigmaminus-0.5*commutator(eta.sigmaminus*eta.sigmaplus, eta))
    eta_new = eta_new + TIMEDELTA * (first_term + second_term)
eta_new

100%|██████████████████████████████████████| 1000/1000 [00:02<00:00, 384.57it/s]


Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = False
Qobj data =
[[270.50302454 +0.j           0.         +0.j
    0.         +0.j        ]
 [  0.         +0.j         135.25151251+77.94315945j
  135.12651251 +0.21650635j]
 [  0.         +0.j         135.12651251 -0.21650635j
  135.25151251-77.94315945j]]

## ETa Dissipator

In [ ]:
from sympy import *

In [ ]:
a, b, j, p = symbols('alpha^2 beta^2 i phi')
g1, g2 = symbols('gamma1 gamma2')
Eta = Matrix([[a, 0, 0], [0, b, b*exp(j*p)], [0, b*exp(-j*p), b]])

In [ ]:
Bp = Matrix([[0, 0, 0], [1, 0, 0], [1, 0, 0]])
Bm = Matrix([[0, 1, 1], [0, 0, 0], [0, 0, 0]])

In [ ]:
g1*(Bm*Eta*Bp-0.5*Bp*Bm*Eta-0.5*Eta*Bp*Bm)+g2*(Bp*Eta*Bm-0.5*Bm*Bp*Eta-0.5*Eta*Bm*Bp)